# Re-entry estimation exercise

This exercise follows the tabular regression model tutorial, but applies it to the problem of predicting values of the re-entry trajectory and ground impact location, based on the uncertanties on initial conditions, object properties and atmospheric characteristics (See Table 1 of the paper attached to the project preview)

The outputs of the regression model will be:
1. Latitude
2. Longitude

Since there are more than one output in the regression model, you can either create a single learner that outputs the two variables, or create two different learners, one to predict each of the outputs. In case you go for the first approach, it’s important to scale your dependent variables before the prediction, otherwise the largest one might dominate everything and you end up optimizing your NN for only one of them. Here are two possible scaling tabular processors (instances of the fastai's `TabularProc` class) that you can introduce when creating the dataloader. You can also do this transformations manually in the input dataframe. 

In [0]:
class NormalizeDep(TabularProc):
    "Normalize the dependent variables of a TabularPandas object"
    order = 3
    def setups(self, dsets): self.y_means,self.y_stds = dsets.ys.mean(),dsets.ys.std(ddof=0)+1e-7
    def encodes(self, to): to.ys = (to.ys-self.y_means) / self.y_stds
    def decodes(self, to): to.ys = (to.ys*self.y_stds ) + self.y_means

In [0]:
class LogDep(TabularProc):
    "Log-transform the dependent variables of a TabularPandas object. If \
    there are negative values in any of the variables, a translation is applied \
    first using the minimum value of that variable plus 1"
    order = 3
    def setups(self, dsets): self.y_log_translation = [abs(x)+1 if x<=0 else 0 for x in list(dsets.ys.min())]
    def encodes(self, to): to.ys = np.log(to.ys + self.y_log_translation)
    def decodes(self, to): to.ys = np.exp(to.ys) -self.y_log_translation

More specifically, the tasks you are expected to do in this exercise are:

1. Create a regression model based on the data that you've previously generated from the black-box propagation model. You should have at least around 200-300 items to create a proper model.

2. Test that model with a big input test set (around 1e6 points), generated from sampling the random input variables.

3. Perform an analysis of the most important features in the regression model.

Remember to use the LTW-I Slack workspace to ask questions and request mentors to your Zoom room. You can also visit the [fastai2 documentation website](https://dev.fast.ai/) for technical troubleshooting if you use fastai2 as ML library. If you are already familiar with any other ML library for training tabular regression models (NNs, tree-based models, SVMs...), you can perfectly use that instead of the one showed in the tutorial, as long as you fulfill the tasks mentioned above.